In [23]:
#!pip install spacy
#!python3 -m spacy download en_core_web_sm
import spacy
#nlp=spacy.load("en_core_web_sm")
#nlp.pipe_names

In [44]:
nlp = spacy.blank('en')
nlp.pipe_names

[]

In [45]:
import en_core_web_sm
#from spacy.pipeline import textcat
from spacy.pipeline.textcat_multilabel import DEFAULT_MULTI_TEXTCAT_MODEL
config = {
   "threshold": 0.5,
   "model": DEFAULT_MULTI_TEXTCAT_MODEL,
}

#nlp = en_core_web_sm.load() # or nlp=spacy.load("en_core_web_sm")

config = Config().from_str(single_label_cnn_config)
if "textcat" not in nlp.pipe_names:
     textcat = nlp.add_pipe('textcat', config=config, last=True)

nlp.pipe_names

['textcat']

In [46]:
# Adding the labels to textcat
textcat.add_label("class1")
textcat.add_label("class2")
textcat.add_label("class3")
textcat.add_label("classneg")

1

In [47]:
train = [
    ("users", 0),
    ("Subscribers",0),
    ("Customers",0),
    ("clients",0),
    ("patrons",0),
    ("payees",0),
    ("members",0),
    ("Average customer lifetime",1),
    ("Customer lifespan",1),
    ("Duration of subscription",1),
    ("Churn",1),
    ("Attrition",1),
    ("Monthly fee",2),
    ("Fee per user",2),
    ("Charge",2),
    ("Cost",2),
    ("Average revenue per user",2),
    ("Total revenue",3),
    ("Recurring revenue",3),
    ("Profits",3),
    ("Annual recurring revenue",3),
    ("Total income",3),
    ("Margins",3),
    ("Monthly",3),
    ("recurring revenue",3)
]

In [48]:
import random

def load_data(limit=0, split=0.9):
    train_data=train
    # Shuffle the data
    random.shuffle(train_data)
    texts, labels = zip(*train_data)
    # get the categories for each review
    cats = [{"class1": y==2, "class2": y==0, "class3": y==1, "classneg": y==3} for y in labels]

    # Splitting the training and evaluation data
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])


# Calling the load_data() function 
(train_texts, train_cats), (dev_texts, dev_cats) = load_data()

# Processing the final format of training data
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))
train_data

[('patrons',
  {'cats': {'class1': False,
    'class2': True,
    'class3': False,
    'classneg': False}}),
 ('clients',
  {'cats': {'class1': False,
    'class2': True,
    'class3': False,
    'classneg': False}}),
 ('Total income',
  {'cats': {'class1': False,
    'class2': False,
    'class3': False,
    'classneg': True}}),
 ('Recurring revenue',
  {'cats': {'class1': False,
    'class2': False,
    'class3': False,
    'classneg': True}}),
 ('Attrition',
  {'cats': {'class1': False,
    'class2': False,
    'class3': True,
    'classneg': False}}),
 ('Customer lifespan',
  {'cats': {'class1': False,
    'class2': False,
    'class3': True,
    'classneg': False}}),
 ('Duration of subscription',
  {'cats': {'class1': False,
    'class2': False,
    'class3': True,
    'classneg': False}}),
 ('payees',
  {'cats': {'class1': False,
    'class2': True,
    'class3': False,
    'classneg': False}}),
 ('users',
  {'cats': {'class1': False,
    'class2': True,
    'class3': False,
    

In [49]:
from spacy.util import minibatch, compounding
from spacy.training.example import Example


n_iter=10

# Disabling other components
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.begin_training()

    # Performing training
    for i in range(n_iter):
        losses = {}
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            example = []
            # Update the model with iterating each text
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)

In [50]:
# Testing the model
test_text="follower"
doc=nlp(test_text)
doc.cats 

{'class1': 0.1205315813422203,
 'class2': 0.6422814726829529,
 'class3': 0.14970755577087402,
 'classneg': 0.08747939765453339}